# 연관규칙 탐색 

- antecedant and consequent

![2_52.png](../materials/2_52.png)
![2_52.png](../materials/2_53.png)

![2_52.png](../materials/2_54.png)

- Support :  A, B가 동시에 발생한 횟수 / 전체 트렌젝션 데이터 크기
- Confidence :  A, B가 동시에 발생한 횟수 / A 발생한 것 가운데

![2_52.png](../materials/2_55.png)
![2_52.png](../materials/2_56.png)

- 아이템집합 격자 
    - 각 `node`는 하나의 아이템 집합을 의미. 
    - 해당 노드에 포함되는 아이템인 경우는 `edge`가 그어진다. 

![2_52.png](../materials/2_57.png)

- 최소 지지도(교집합/전체)는 사용자가 정의하는 지지도. 이 규칙을 넘어서면 frequent한 것. 
- `Apriori 원리`: 어떤 아이템 집합이 frequent 하면(즉, minimum support 이상이면), 그 아이템의 부분집합 또한 frequent하다. 
    - 즉, 빵과 우유의 집합이 빈발하면 빵과 우유는 각각 둘다 빈발하다. 
    - 하단 증명 보면 너무 당연함. 

![2_52.png](../materials/2_58.png)

- `AB`집합이 not frequent한 경우, 그 부모의 집합도 빈발하지 않다. 
    - 위 증명에 대한 대우 명제. 
    - `대우명제` : **부분 집합이 not frequent -> 그 부모의 집합도 not frequent**

- 즉, 아랫 단계에서 AB가 not frequent가 판단되는 순간 그 부모 집합들 싹다 배재하면 되는 것. 효율적인 탐색이 가능하다. 


![2_52.png](../materials/2_59.png)

조금 더 말로 풀어보자면 다음과 같다. 
- 최대 빈발 아이템 집합 : 최소 지지도 이상이면서, 이 집합의 모든 부모들은 not frequent한 집합. 
    - 즉, {A, B, C}는 frequent한데, {A, B, C, D}. {A, B, C, E}, {A, B, C, D. E} 등 그 부모가 될 수 있는 모든 집합이 not frequent
    - 그렇다면, {A, B, C}는 최대빈발아이템집합
- {A, B, C}는 frequent하기 때문에 {A, B, C}의 부분집합들은 싹다 Frequent. 
- 이제 그 A, B, C를 가지고 부모와 자식이 될 수 있는 모든 경우의 수를 따진 것이 아래 테이블과 같다. 


![2_52.png](../materials/2_60.png)

**Confidence** :  A, B가 동시에 발생한 횟수 / A 발생한 것 가운데
- 그렇다면 신뢰도에 대한 Apriori 원리는?
- **동일한 아이템 집합으로 만든 규칙**. X1이 X2에 속하게 되면, `C(X1 -> Y1) <= C(X2 -> Y2)`가 성립한다. 

![2_52.png](../materials/2_61.png)

- 모두가 같은 아이템 집합에서 온 경우 
- 맨 위부터 부모에 가능한 한 많이 넣는 경우부터 출발. 
- A와 B는 {A, B}의 부분집합 이기 때문에, `A-> `, `B->`의 신뢰도는 반드시 `{A, B} -> `보다 낮을 수 밖에 없다. 

![2_52.png](../materials/2_62.png)

**즉, 정리하자먄, 지지도에 대한 Apriori원리는 최대빈발아이템 집합을 찾는데 사용하고, 신뢰도에 대한 Apriori 원리는 최대빈발아이템 집합으로 부터 만들어지는 규칙들 가운데 최소신뢰도 이하인 것을 탐색하는데 활용한다. **


패키지가 있으나, 직접 구현해도 그렇게 어렵진 않다. 빈발 아이템 규칙을 먼저 찾아내고 연관 규칙을 찾아내는 **두 단계**에 걸쳐 수행한다. 

- apiori 함수 : 빈발 아이템 집합을 탐색하는데 사용한다. 
- association rules 함수 : 연관규칙 탐색하는데 사용된다.
![2_52.png](../materials/2_63.png)

- 연관규칙 탐사에 적절하게 거래 데이터 구조를 바꾸기 위한 함수이다. 
- 인스턴스 생성 후, 원본 data를 각 아이템의 출현 여부를 같는 ndarray 및 DataFrame으로 변환해준다. 
![2_52.png](../materials/2_64.png)

In [11]:
import os
import pandas as pd

os.chdir(r"/Users/sanghyuk/Documents/preprocessing_python/lecture_source/2. 탐색적 데이터 분석/데이터")


import pandas as pd
df = pd.read_csv("Instacart Market Basket Analysis.csv")

In [12]:
df.head()

# order_id = 1번 주문에 대해서, product_id에 해당되는 물건들이 주문된 것. 

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [13]:
# 잘 보면, 비율 자체가 그렇게 높지 않음. 
# 다른 데이터들도 대부분 그렇다. 

df['product_id'].value_counts(normalize = True).head(100)

24852    0.013524
13176    0.011180
21137    0.007868
21903    0.007066
47626    0.005875
           ...   
8193     0.001024
9387     0.000996
37687    0.000984
20995    0.000983
34243    0.000976
Name: product_id, Length: 100, dtype: float64

현재 위 데이터는 각 Order_id별로, Product_id가 각각 한 row씩 먹고 있어서, 데이터 분석으로 부적절하다. <br>
데이터 구조를 바꿔 줘야 함. 

In [14]:
# group by, apply(list) 자주 쓰네. 

product_list_per_order = df.groupby('order_id')['product_id'].apply(list)

In [15]:
product_list_per_order

order_id
1          [49302, 11109, 10246, 49683, 43633, 13176, 472...
36         [39612, 19660, 49235, 43086, 46620, 34497, 486...
38         [11913, 18159, 4461, 21616, 23622, 32433, 2884...
96         [20574, 30391, 40706, 25610, 27966, 24489, 39275]
98         [8859, 19731, 43654, 13176, 4357, 37664, 34065...
                                 ...                        
3421049           [40800, 17706, 33424, 17299, 26800, 34243]
3421056                    [5750, 9340, 21709, 16475, 12432]
3421058    [15629, 4347, 34466, 6244, 6858, 30316, 35578,...
3421063                         [49235, 13565, 14233, 35548]
3421070                                 [35951, 16953, 4724]
Name: product_id, Length: 131209, dtype: object

In [7]:
# !pip3 install mlxtend

In [16]:
len(np.unique(df['product_id']))

39123

In [19]:
print(len(one_hot_df.index))
print(len(np.unique(df['order_id'])))

131209
131209


In [18]:
# 구매 기록 데이터 -> One hot encoding 데이터
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder() # 인스턴스화

# columns are product_id
one_hot_df = encoder.fit(product_list_per_order).transform(product_list_per_order) # 결과: ndarray
one_hot_df = pd.DataFrame(one_hot_df, columns = encoder.columns_)
one_hot_df.tail() # 매우 희소 (sparse)

,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131208,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
from mlxtend.frequent_patterns import *

frequent_item_df = apriori(one_hot_df, min_support = 0.003) # 0.3% 이상 구매한 상품만 대상으로 함, 실제로는 이 정도 숫자도 흔치가 않다. 보통은 1%도 안된다. 
# confidence 0.1 이상인 연관규칙을 뽑아내겠다. 
result = association_rules(frequent_item_df, metric = 'confidence', min_threshold = 0.1)
result[['antecedents', 'consequents', 'support', 'confidence']].sort_values(by = 'confidence', ascending = False).to_csv("연관규칙탐사결과.csv", index = False)

In [ ]:
frequent_item_df

여러가지 리포트 지표들이 보인다. 
- antecedent, consequent의 Support를 각각 보여준다. 

In [9]:
result.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(12022),(1815),0.014206,0.008887,0.003018,0.212446,23.906409,0.002892,1.258471
1,(1815),(12022),0.008887,0.014206,0.003018,0.339623,23.906409,0.002892,1.492773
2,(3640),(17300),0.028672,0.074568,0.004146,0.144604,1.939221,0.002008,1.081875
3,(3640),(19574),0.028672,0.142719,0.008163,0.284689,1.994754,0.004071,1.198473
4,(3640),(20648),0.028672,0.045980,0.003780,0.131845,2.867432,0.002462,1.098905
